In [1]:
import pandas as pd
import re

pd.set_option('display.max_columns', 75)

## Loading crimes data

Assumptions about the underlying data:
- It has a hierarchical index
- It's not fully structured but contains metadata, as one could do in an Excel file

In [2]:
read_csv_common_kwargs = {
    'filepath_or_buffer': '../data/input/delitos_por_municipio.csv',
    'sep': ';',
    'index_col': 0,
    'skiprows': 4,
    'encoding': 'latin-1'
}

df_crimes_raw = pd.read_csv(**read_csv_common_kwargs, header = [0, 1], skipfooter = 7, engine = 'python')

# drop null cols
cols_bool_mask = df_crimes_raw.isna().all(axis = 0)
df_crimes = df_crimes_raw.drop(columns = df_crimes_raw.columns[cols_bool_mask])

# set columns as multi-index
headers = pd.read_csv(**read_csv_common_kwargs, header = None, nrows = 2, keep_default_na = True)
header_levels = [row[row.notna()].unique() for _, row in headers.iterrows()]
df_crimes.columns = pd.MultiIndex.from_product(header_levels)

# prettify column names
df_crimes.rename(columns = lambda x: re.sub('(\d+\.)+-', '', x), inplace = True)

# simplify municipality name
df_crimes.index = df_crimes.index.str.replace('- Municipio de | \(Las\)', '', regex = True)

print(df_crimes.shape)
df_crimes.head()

(28, 30)


Homicidios dolosos y asesinatos consumados  \
                                                Enero-marzo 2019   
MADRID (COMUNIDAD DE)                                       10.0   
Alcalá de Henares                                            1.0   
Alcobendas                                                   0.0   
Alcorcón                                                     0.0   
Aranjuez                                                     0.0   

                                        \
                      Enero-marzo 2020   
MADRID (COMUNIDAD DE)             11.0   
Alcalá de Henares                  0.0   
Alcobendas                         0.0   
Alcorcón                           0.0   
Aranjuez                           0.0   

                      Homicidios dolosos y asesinatos en grado tentativa  \
                                                        Enero-marzo 2019   
MADRID (COMUNIDAD DE)                                               15.0   
Alcalá de Henares                                                    0.0   
Alcobendas                                                           1.0   
Alcorcón                                                             1.0   
Aranjuez                                                             0.0   

                                        \
                      Enero-marzo 2020   
MADRID (COMUNIDAD DE)             19.0   
Alcalá de Henares                  1.0   
Alcobendas                         1.0   
Alcorcón                           0.0   
Aranjuez                           0.0   

                      Delitos graves y menos graves de lesiones y riña tumultuaria  \
                                                                  Enero-marzo 2019   
MADRID (COMUNIDAD DE)                                              598.0             
Alcalá de Henares                                                   18.0             
Alcobendas                                                           9.0             
Alcorcón                                                            10.0             
Aranjuez                                                             4.0             

                                              Secuestro                   \
                      Enero-marzo 2020 Enero-marzo 2019 Enero-marzo 2020   
MADRID (COMUNIDAD DE)            576.0              3.0              2.0   
Alcalá de Henares                 10.0              0.0              1.0   
Alcobendas                         7.0              0.0              0.0   
Alcorcón                          11.0              0.0              0.0   
Aranjuez                           5.0              0.0              0.0   

                      Delitos contra la libertad e indemnidad sexual  \
                                                    Enero-marzo 2019   
MADRID (COMUNIDAD DE)                                          539.0   
Alcalá de Henares                                               10.0   
Alcobendas                                                       6.0   
Alcorcón                                                         8.0   
Aranjuez                                                         6.0   

                                       Agresión sexual con penetración  \
                      Enero-marzo 2020                Enero-marzo 2019   
MADRID (COMUNIDAD DE)            502.0                            56.0   
Alcalá de Henares                 12.0                             1.0   
Alcobendas                        10.0                             0.0   
Alcorcón                           8.0                             4.0   
Aranjuez                           0.0                             1.0   

                                        \
                      Enero-marzo 2020   
MADRID (COMUNIDAD DE)             58.0   
Alcalá de Henares                  0.0   
Alcobendas                         2.0   
Alcorcón                           1.0   
Aranjuez  

## Loading income data

Assumptions
- `Total` column is the value of the given metric, and thus it should be numeric
- `Municipios` column is a combination of the zip code and the municipality name

In [3]:
df_income_raw = pd.read_csv('../data/input/renta_por_hogar.csv', sep = ';')
df_income_raw['Total'] = pd.to_numeric(df_income_raw['Total'], errors = 'coerce')

# pivot data
df_income = df_income_raw.pivot_table(index = 'Municipios', columns = ['Indicadores de renta media y mediana', 'Periodo'], values = 'Total')

# note: before computing zip code and municipality name we could flatten the column multi-index
df_income[['CP','Municipio']] = df_income.index.str.extract('(\d{5}) ([\w\s]+)(?:,\w+)?', expand = True).values.tolist()
df_income['CP'] = pd.to_numeric(df_income['CP'])

# remove column index names and drop municipality (row) index
df_income.columns.names = (None, None)
df_income = df_income.reset_index(drop = True).reindex(columns = df_income.columns[-2:].tolist() + df_income.columns[:-2].tolist())

print(df_income.shape)
df_income.head(5)

(179, 38)


CP          Municipio Media de la renta por unidad de consumo  \
                                                               2015   
0  28001            Acebeda                                     NaN   
1  28002            Ajalvir                               19.316000   
2  28003  Alameda del Valle                               18.428000   
3  28004              Álamo                               14.280500   
4  28005  Alcalá de Henares                               17.142351   

                                                          \
        2016       2017       2018       2019       2020   
0        NaN        NaN        NaN        NaN        NaN   
1  18.907000  20.189250  21.027250  22.176750  21.831000   
2  18.958000  19.858000  19.965000  20.675000  22.107000   
3  14.559667  15.043167  15.563833  16.180500  16.506333   
4  17.097191  18.167885  18.940206  19.636038  19.708458   

  Mediana de la renta por unidad de consumo                                   \
                                       2015       2016       2017       2018   
0                                       NaN        NaN        NaN        NaN   
1                                 16.625000  16.625000  18.025000  18.550000   
2                                 15.750000  15.750000  17.850000  17.150000   
3                                 12.833333  12.950000  13.533333  14.233333   
4                                 15.624427  15.581679  16.564885  17.393130   

                        Renta bruta media por hogar                        \
        2019       2020                        2015       2016       2017   
0        NaN        NaN                         NaN        NaN        NaN   
1  19.250000  18.550000                   42.976750  43.201500  44.582250   
2  16.450000  19.950000                   33.471000  34.743000  35.168000   
3  14.816667  14.933333                   30.967333  31.944000  32.865167   
4  18.133206  18.178626                   37.984557  38.674733  39.829160   

                                   Renta bruta media por persona             \
        2018       2019       2020                          2015       2016   
0        NaN        NaN        NaN                           NaN        NaN   
1  46.597250  49.692750  48.867500                     15.190000  14.889250   
2  34.921000  36.851000  37.622000                     16.980000  17.898000   
3  34.086000  35.785000  36.566333                     10.559167  10.769167   
4  41.532183  43.124282  42.971099                     13.321931  13.257847   

                                              Renta neta media por hogar  \
        2017       2018       2019       2020                       2015   
0        NaN        NaN        NaN        NaN                        NaN   
1  15.803000  16.410000  17.468750  17.297750                  35.015750   
2  18.840000  18.695000  19.424000  20.658000                  27.355000   
3  11.154167  11.615833  12.121833  12.437000                  26.683000   
4  14.220206  14.884840  15.485260  15.626893                  31.922313   

                                                          \
        2016       2017       2018       2019       2020   
0        NaN        NaN        NaN        NaN  27.408000   
1  35.206250  36.563500  38.236250  40.678750  39.894750   
2  28.416000  28.783000  28.304000  30.053000  30.463000   
3  27.467333  28.173000  29.163833  30.646167  31.180000   
4  32.462771  33.327053  34.649908  35.941985  35.786985   

  Renta neta media por persona                                               \
                           2015       2016       2017       2018       2019   
0                           NaN        NaN        NaN        NaN        NaN   
1                     12.377250  12.133000  12.961000  13.465500  14.298750   
2                     13.878000  14.638000  15.419000  15.153000  15.841000   
3                      9.097167   9.259500   9.562000   9.938500  10.381167   
4   

## Loading contact center data

Assumptions
- The underlying data corresponds to some events table, hence the `DNI`, `Telef`, `CP` and `duration_call_mins` columns have the same value for a given `sessionID`
- The session ID should not be null, and if it is the corresponding data could not be reliable.

In [4]:
df_cc_raw = pd.read_csv(
    '../data/input/contac_center_data.csv',
    sep = ';'
)

df_cc = df_cc_raw.groupby(
    by = ['sessionID', 'DNI', 'Telef', 'CP', 'duration_call_mins'],
    as_index = False,
    dropna = True
).agg(
    funnel_steps = ('funnel_Q', list),
    product = ('Producto', lambda x: pd.unique(x)[0])
)

# add a dummy multi-index to simplify the juntion with the other data
df_cc.columns = pd.MultiIndex.from_tuples([(col, '') for col in df_cc.columns])

print(df_cc.shape)
df_cc.head()

(9076, 7)


,sessionID,DNI,Telef,CP,duration_call_mins,funnel_steps,product
,,,,,,,
0,b'MC40MDA1NDM0MzY5MzYyODE4',X88537286,671483480,28520,13.481688,"[Chalet, Unifamiliar, Sin Rejas]",Home Premium Plus
1,b'MC40MDA2OTkxNDAyNzA5Nzcz',X67868023,669613626,28758,7.066443,"[Chalet, Adosado, Con Rejas]",NaN
2,b'MC40MDA5Mzc0NTc5MzM5OTAxNg==',X22379422,688974839,28001,14.197395,"[Piso, Bajo, Con Rejas, Con Perro]",Home Premium
3,b'MC40MDA5MzgwNDg5NzU2ODM2NA==',X95888915,604709311,28678,6.940735,"[Piso, Bajo, Con Rejas, Sin Perro]",NaN
4,b'MC40MDA5NjA1MDcwNjY2MzQ1NQ==',X76422588,610413816,28881,5.674006,"[Piso, Bajo, Sin Rejas, Sin Perro]",NaN


## Merging dataframes

Assumptions
- For the project analytical purpose, we need data from all sources, thus we use inner joins when merging.

Note the final dataframe has a non-lex-sorted multi-index, which affects performance on slicing and lookup operations. To overcome this problem we can either sort or flatten the columns multi-index, but that might hinder data visualization so I have not done it. Flattenning the multi-index could be done as follows
```python
df_all.columns = (
    df_all.columns
        .set_levels(df_all.columns.levels[1].astype(str), level = 1)
        .to_flat_index()
        .map(' '.join)
        .str.strip()
)
```

In [5]:
df_tmp = df_income.merge(df_crimes, how = 'inner', left_on = 'Municipio', right_index = True)
df_all = df_cc.merge(df_tmp.set_index('CP'), how = 'inner', left_on = 'CP', right_index = True).reset_index(drop = True)

print(df_all.shape)
df_all.head(10)

(241, 74)


sessionID        DNI      Telef     CP  \
                                                              
0  b'MC40MDIwMTkxMTQwNDQ5Mjc1'  X84384073  610706856  28092   
1  b'MC40ODI1OTgyODQ2ODEwMjY2'  X65339116  635462198  28092   
2  b'MC41NTE1ODg1ODc5NDU4MTQ0'  X84384073  610706856  28092   
3  b'MC41NzkzNTEzNzY1ODcwOTU4'  X67376025  624354169  28092   
4  b'MC42MDQxMjM5NTA2NTY5NDc5'  X67376025  624354169  28092   
5  b'MC42NDA3NTk0MDk1MDc5NA=='  X65339116  635462198  28092   
6  b'MC42NTUwOTc1NjgxNzAxNzY='  X84384073  610706856  28092   
7  b'MC42Njc0MTQxMzgxNDc5NDc1'  X84384073  610706856  28092   
8  b'MC42NzA3NDg4Njk5NTQ3MDgz'  X16558297  629922565  28092   
9  b'MC42OTE1Mzg5NDE3MTE5NTYz'  X16558297  629922565  28092   

  duration_call_mins                        funnel_steps       product  \
                                                                         
0           6.185554       [Piso, Intermedio, Con Perro]    Home Basic   
1          18.208040    [Chalet, Unifamiliar, Con Rejas]  Home Premium   
2          17.707779       [Piso, Intermedio, Con Perro]    Home Basic   
3          18.092605  [Piso, Bajo, Con Rejas, Con Perro]  Home Premium   
4          17.200239  [Piso, Bajo, Con Rejas, Con Perro]  Home Premium   
5          23.387328    [Chalet, Unifamiliar, Con Rejas]  Home Premium   
6          23.898185       [Piso, Intermedio, Con Perro]    Home Basic   
7          28.866454       [Piso, Intermedio, Con Perro]    Home Basic   
8           3.845364  [Piso, Bajo, Con Rejas, Sin Perro]           NaN   
9           2.028235  [Piso, Bajo, Con Rejas, Sin Perro]           NaN   

  Municipio Media de la renta por unidad de consumo                        \
                                               2015       2016       2017   
0  Móstoles                               15.923358  16.201212  16.713284   
1  Móstoles                               15.923358  16.201212  16.713284   
2  Móstoles                               15.923358  16.201212  16.713284   
3  Móstoles                               15.923358  16.201212  16.713284   
4  Móstoles                               15.923358  16.201212  16.713284   
5  Móstoles                               15.923358  16.201212  16.713284   
6  Móstoles                               15.923358  16.201212  16.713284   
7  Móstoles                               15.923358  16.201212  16.713284   
8  Móstoles                               15.923358  16.201212  16.713284   
9  Móstoles                               15.923358  16.201212  16.713284   

                                   Mediana de la renta por unidad de consumo  \
        2018       2019       2020                                      2015   
0  17.405099  18.141752  18.158113                                 14.763869   
1  17.405099  18.141752  18.158113                                 14.763869   
2  17.405099  18.141752  18.158113                                 14.763869   
3  17.405099  18.141752  18.158113                                 14.763869   
4  17.405099  18.141752  18.158113                                 14.763869   
5  17.405099  18.141752  18.158113                                 14.763869   
6  17.405099  18.141752  18.158113                                 14.763869   
7  17.405099  18.141752  18.158113                                 14.763869   
8  17.405099  18.141752  18.158113                                 14.763869   
9  17.405099  18.141752  18.158113                                 14.763869   

                                                     \
    2016       2017       2018      2019       2020   
0  15.05  15.613475  16.313475  17.04078  16.991135   
1  15.05  15.613475  16.313475  17.04078  16.991135   
2  15.05  15.613475  16.313475  17.04078  16.991135   
3  15.05  15.613475  16.313475  17.04078  16.991135   
4  15.05  15.613475  16.313475  17.04078  16.991135   
5  15.05  15.613475  16.313475  17.04078  16.991135   
6  15.05  15.613475  16.313475  17.04078  16.9911